In [ ]:
import os

In [ ]:
%pwd

'c:\\Users\\Hp\\Downloads\\Kidney_Disease_Classification\\research'

In [ ]:
os.chdir("../")

In [31]:
import os
import mlflow

# Set MLflow tracking credentials for DagsHub
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/trilochanp3/Kidney_Disease_Classifaction_Deep_Learning_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "trilochanp3"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "92afea69a0575927645cf630261c4834d29ee712"

# Set tracking URI explicitly
mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])

# Verify MLflow connection
print("Tracking URI:", mlflow.get_tracking_uri())

import dagshub
dagshub.init(repo_owner='trilochanp3', repo_name='Kidney_Disease_Classifaction_Deep_Learning_project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Tracking URI: https://dagshub.com/trilochanp3/Kidney_Disease_Classifaction_Deep_Learning_project.mlflow


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=4c3332ff-f3c0-4179-b494-9a3cee8470b9&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=56edd225dfd8dd7974ffac72bfabc8014130e9b33498f67395556354101c0130


[ 2025-03-26 14:39:53,512: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[ 2025-03-26 14:39:53,933: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[ 2025-03-26 14:39:54,299: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as trilochanp3

[ 2025-03-26 14:39:54,313: INFO: helpers: Accessing as trilochanp3]
[ 2025-03-26 14:39:54,710: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/trilochanp3/Kidney_Disease_Classifaction_Deep_Learning_project "HTTP/1.1 200 OK"]
[ 2025-03-26 14:39:55,077: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "trilochanp3/Kidney_Disease_Classifaction_Deep_Learning_project"

[ 2025-03-26 14:39:55,083: INFO: helpers: Initialized MLflow to track repo "trilochanp3/Kidney_Disease_Classifaction_Deep_Learning_project"]


Repository trilochanp3/Kidney_Disease_Classifaction_Deep_Learning_project initialized!

[ 2025-03-26 14:39:55,083: INFO: helpers: Repository trilochanp3/Kidney_Disease_Classifaction_Deep_Learning_project initialized!]


In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [37]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_dierctories, save_json

In [33]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_dierctories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [34]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [38]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()

except Exception as e:
    raise e

[ 2025-03-26 14:46:08,311: INFO: common: yaml file: config\config.yaml]
[ 2025-03-26 14:46:08,314: INFO: common: yaml file: params.yaml]
[ 2025-03-26 14:46:08,316: INFO: common: create dirstory at : artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 12s 1s/step - loss: 0.0808 - accuracy: 0.9568
[ 2025-03-26 14:46:20,746: INFO: common: json data saved at : scores.json]
